In [18]:
import pandas as pd
import numpy as np
import json
import os
import datetime
import requests
import tarfile
import sys
import csv
import importlib

# Add the parent directory to the system path so the notebook can find the eBird2ABAP package
sys.path.append(os.path.abspath("../"))

import eBird2ABAP

# Reload the module to apply changes
# importlib.reload(eBird2ABAP.ebird2card)

<module 'eBird2ABAP.ebird2card' from '/Users/rafnuss/Library/CloudStorage/OneDrive-Vogelwarte/eBird2ABAP/eBird2ABAP/ebird2card.py'>

In [2]:
year = "2024"
month = "Jul"

# file = "../data/eBird/ebd_AFR_rel{month}-{year}/ebd_AFR_rel{month}-{year}.txt.gz"
file = f"../data/eBird/ebd_AFR_rel{month}-{year}/ebd_AFR_rel{month}-{year}.txt.gz"
# file = "../data/eBird/ebd_AFR_relJul-2024/ebd_AFR_relJul-2024.txt.gz"

ebd = read_EBD(file)

In [3]:
add_ADU(ebd, return_unmatched=True)

We still have 1260 unmatched taxons, corresponding to 7% of our data


SCIENTIFIC NAME        
Icthyophaga vocifer        90635
Spermestes cucullata       82830
Columba livia              62539
Zapornia flavirostra       51197
Lanius melanoleucus        34528
                           ...  
Chamaea fasciata               1
Chalcoparia singalensis        1
Chaetura pelagica              1
Turnix velox                   1
Sicalis luteola                1
Name: count, Length: 1260, dtype: int64

In [4]:
ebd = add_ADU(ebd)
print(f"# data: {len(ebd)}")
print(f"# Checklists: {len(ebd['SAMPLING EVENT IDENTIFIER'].unique())}")
print(f"# Species: {len(ebd['SCIENTIFIC NAME'].unique())}")

# data: 21190167
# Checklists: 1262874
# Species: 3127


In [5]:
chk = ebd2chk(ebd)

In [6]:
card_valid = chk2valid_card(chk)

In [7]:
chk_card = valid_card2chk_card(chk, card_valid)

In [19]:
card_chk = chk_card2card_chk(chk_card, card_valid)

In [20]:
ebd_f_u = chk_card2ebd_f_u(ebd, chk_card)

In [21]:
card_exp = ebd_f_u2card_exp(card_chk, ebd_f_u)

In [22]:
len(card_exp)

62396

In [23]:
json_data = card_exp.to_json(orient="records", indent=2)
with open(
    f"../export/ebd_AFR_rel{month}-{year}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
    "w",
) as f:
    f.write(json_data)

In [ ]:
card_chk.to_csv(
    f"../export/ebd_AFR_rel{month}-{year}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_cards.csv",
    index=False,
)

In [ ]:
ebd_f_u[["CARD", "ADU", "SEQ"]].to_csv(
    f"../export/ebd_AFR_rel{month}-{year}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_records.csv",
    index=False,
)